In [1]:
import polars as pl

In [3]:

from humbldata.toolbox.technical.mandelbrot_channel.model import (
    calc_mandelbrot_channel,
    calc_mandelbrot_channel_historical,
)

mandelbrot = calc_mandelbrot_channel_historical(
        data,
        window="1m",
        rv_adjustment=True,
        rv_method="std",
        rv_grouped_mean=False,
        rs_method="RS",
        live_price=False,
    ).collect()

In [10]:
mandelbrot

date,symbol,bottom_price,recent_price,top_price
date,str,f64,f64,f64
2023-12-29,"""AAPL""",191.0641,192.284637,214.0328
2023-12-29,"""AMD""",131.4548,147.410004,152.316
2023-12-29,"""AMZN""",146.3461,151.940002,159.3488
2023-12-29,"""GOOGL""",135.1156,139.529861,141.2999
2023-12-29,"""PCT""",2.4087,4.05,5.1265
